<a href="https://colab.research.google.com/github/PabloMendieta03/UFV-VisualizacionDeDatos/blob/main/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

Python 3.10.12


# Ejercicio de histórico de ventas: Planificación

## Carga de la Base de Datos

In [ ]:
# Importación de librerías
import pandas as pd
from google.colab import drive
import plotly.express as px
from sklearn.metrics import mean_squared_error # calcular el error cuadrático medio

In [ ]:
drive.mount('/content/drive')

# Especicicamos la ruta del archivo CSV en Google Drive
ruta_archivo_drive = '/content/drive/MyDrive/datos_ejercicio_ventas.csv'

# Cargamos el archivo CSV en un DataFrame de pandas
data = pd.read_csv(ruta_archivo_drive)

# Crear la columna de fecha
data['DATE'] = pd.to_datetime(data[['YEAR', 'MONTH']].assign(DAY=1))

# mostrar los datos
display(data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT,DATE
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,7.543562e+05,2023-12-01
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,5.600306e+05,2023-12-01
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,8.850198e+04,2023-12-01
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,3.632245e+05,2024-12-01
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,3.961761e+05,2023-09-01
...,...,...,...,...,...,...,...,...,...
18661,Great Britain,Pepsi Regular (L3),2024,2,AI_forecast,AI_P10F,2023.0,1.313511e+06,2024-02-01
18662,Hungary,Pepsi Regular (L3),2024,7,AI_forecast,AI_P07F,2023.0,1.314395e+06,2024-07-01
18663,Norway,7up (L3),2024,1,AI_forecast,AI_P05F,2023.0,0.000000e+00,2024-01-01
18664,Portugal,Lipton (L3),2024,3,AI_forecast,AI_P02F,2023.0,5.330634e+05,2024-03-01


## Análisis de los Datos

In [ ]:
# Eliminar duplicados
data = data.drop_duplicates()

# Contar valores negativos en la columna 'AMOUNT'
valores_negativos = data[data['AMOUNT'] < 0].shape[0]
print("Número de valores negativos en la columna AMOUNT:", valores_negativos)

# No tiene sentido trabajar con valores de 'AMOUNT' negativos
data = data[data['AMOUNT'] > 0]

print("Valores negativos eliminados. Número total de filas ahora:", data.shape[0])

Número de valores negativos en la columna AMOUNT: 10
Valores negativos eliminados. Número total de filas ahora: 17302


### **Variable 'SCENARIO'**



In [ ]:
# Obtener las categorías únicas en 'SCENARIO'
categorias = data['SCENARIO'].unique()
print("Categorías en 'SCENARIO':", categorias)

Categorías en 'SCENARIO': ['AI_forecast' 'actual']


¿ Número de acutals y AI_forecast ?

In [ ]:
# Contar los registros de escenario
ActualsForecast = data['SCENARIO'].value_counts()

# Calcular porcentajes
total = ActualsForecast.sum()
porcentajes = (ActualsForecast / total) * 100

# Gráfico de Barras
fig = px.bar(porcentajes,
             x=porcentajes.values,
             y=porcentajes.index,
             orientation='h', # Barras en horizontal
             labels={'y': 'Escenario', 'x': 'Porcentaje (%)'},
             title='Distribución de Actual vs AI_forecast en Porcentajes',
             text=porcentajes.values  # Valores en las barras
)

# Ajustar el ancho de las barras
fig.update_traces(width=0.3)

# Truncar los porcentajes y poner %
fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside')

# Mostrar el gráfico
fig.show()



### **Varible 'COUNTRY'**

¿ Número de Países ?

In [ ]:
# Obtener el número de países únicos
paises = data['COUNTRY'].nunique()
print(f"El número de países es: {paises}")

El número de países es: 9


In [ ]:
# Contar los registros de COUNTRY
VacesPais = data['COUNTRY'].value_counts()

# Calcular porcentajes
totalPais = VacesPais.sum()
porcentajesPais = (VacesPais / totalPais) * 100

# Gráfico de Pastel
fig = px.pie(
    names=porcentajesPais.index, # Etiquetas
    values=porcentajesPais.values, # Valores númericos
    title='Distribución de veces que aparece cada país',
    labels={'values': 'Porcentaje (%)', 'names': 'País'},
    hole=0.3, # Pie Chart tipo Donut
    color_discrete_sequence=px.colors.sequential.Reds_r # Secuencias de colores
)

# Mostrar porcentajes
fig.update_traces(textinfo='percent')

# Mostrar el gráfico
fig.show()

### **Variable 'SUBBRAND'**

¿ Número de Productos ?

In [ ]:
# Obtener el número de productos únicos
productos = data['SUBBRAND'].nunique()
print(f"El número de productos diferentes es: {productos}")

El número de productos diferentes es: 6


In [ ]:
# Contar los registros de SUBBRAND
VacesProducto = data['SUBBRAND'].value_counts()

# Calcular porcentajes
totalProducto = VacesProducto.sum()
porcentajesProducto = (VacesProducto / totalProducto) * 100

# Gráfico de Pastel
fig = px.pie(
    names=porcentajesProducto.index,
    values=porcentajesProducto.values,
    title='Distribución de veces que aparece cada país',
    labels={'values': 'Porcentaje (%)', 'names': 'País'},
    hole=0.3,
    color_discrete_sequence=px.colors.qualitative.Set3
)

# Mostrar porcentajes
fig.update_traces(textinfo='percent')

fig.show()

### **Variable 'FORECAST'**

In [ ]:
# Obtener el número de forecasts distintos
forecastsDistintos = data['FORECAST'].nunique()
print(f"El número de forecasts distintos es: {forecastsDistintos}")

El número de forecasts distintos es: 12


### **Histórico de Datos: Actuals/Forecast**

In [ ]:
# Agrupar por fecha y escenario, sumando el amount
ActualForecastData = data.groupby(['DATE', 'SCENARIO'], as_index=False)['AMOUNT'].sum()

# Graficar
fig = px.line(ActualForecastData, x='DATE', y='AMOUNT',
              color='SCENARIO',
              title='Histórico de Datos: Actuals vs Forecast',
              labels={'AMOUNT': 'Cantidad', 'DATE': 'Fecha'})


fig.show()

### **Horizontes de Previsión:**

In [ ]:
datosUnicos = data.drop_duplicates(subset=['COUNTRY', 'SUBBRAND', 'FORECAST', 'FORECAST_YEAR', 'DATE'])

frecuencias = datosUnicos.groupby(['COUNTRY', 'SUBBRAND', 'FORECAST', 'FORECAST_YEAR']).size().value_counts().reset_index(name='NUMERO_FRECUENCIAS_UNICAS')
print(frecuencias)

    index  NUMERO_FRECUENCIAS_UNICAS
0      18                        512
1      16                          7
2      11                          4
3      17                          3
4      15                          2
5      14                          2
6       9                          2
7       1                          2
8       7                          1
9      13                          1
10     10                          1
11      5                          1


## 1. Distribución de las ventas Realizadas:



### a) Por Países:

In [ ]:
# Agrupar por país y escenario, sumando las ventas
ventasPais = data.groupby(['COUNTRY', 'SCENARIO'])['AMOUNT'].sum().reset_index()

# Filtrar solo para el escenario 'actual'
ventasPais_actual = ventasPais[ventasPais['SCENARIO'] == 'actual']

# Ordenar los países de mayor a menor según las ventas
ventasPaisTotales = ventasPais_actual.sort_values(by='AMOUNT', ascending=False)

# Unir las ventas por país y escenario con las ventas totales
ventasPais_actual['COUNTRY'] = pd.Categorical(ventasPais_actual['COUNTRY'], categories=ventasPaisTotales['COUNTRY'], ordered=True)
ventasPais_actual = ventasPais_actual.sort_values('COUNTRY')

# Gráfico de Barras
fig = px.bar(ventasPais_actual,
             x='COUNTRY',
             y='AMOUNT',
             title='Distribución de Ventas Realizadas por País',
             labels={'COUNTRY': 'País', 'AMOUNT': 'Ventas Totales'},
             text_auto=True
)

fig.show()

<ipython-input-106-eada3ee018ef>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### b) Por mes y año:

In [ ]:
# Filtrar solo las filas donde SCENARIO es 'actual'
data_actual = data[data['SCENARIO'] == 'actual']

# Agrupamos por mes y año y sumamos las ventas
ventas_por_mes = data_actual.groupby(data['DATE'].dt.to_period('M'))['AMOUNT'].sum().reset_index()

# Convertir a formato de fecha
ventas_por_mes['DATE'] = ventas_por_mes['DATE'].dt.to_timestamp()

# Gráfico de Líneas
fig = px.line(ventas_por_mes, x='DATE', y='AMOUNT',
              title='Distribución de Ventas Realizadas por Mes y Año ',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True
)

fig.show()

In [ ]:
# Agrupamos por país, mes y año y sumamos las ventas
ventas_por_mes = data_actual.groupby(['COUNTRY', data['DATE'].dt.to_period('M')])['AMOUNT'].sum().reset_index()

# Ordenar de mayor a menor según las ventas
ventas_por_mes = ventas_por_mes.sort_values(by='AMOUNT', ascending=False)

# Convertir a formato de fecha
ventas_por_mes['DATE'] = ventas_por_mes['DATE'].dt.to_timestamp()

# Gráfico de Área
fig = px.area(ventas_por_mes, x='DATE', y='AMOUNT',
              title='Distribución de Ventas por Mes y Año por País',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              color='COUNTRY'
)

fig.show()

### c) Por marca:

In [ ]:
# Agrupamos por submarca y sumamos las ventas
ventas_por_submarca = data_actual.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()

# Gráfico de Pastel
fig = px.pie(ventas_por_submarca, values='AMOUNT', names='SUBBRAND',
             title='Distribución de Ventas por Marca',
             labels={'SUBBRAND': 'Marca', 'AMOUNT': 'Ventas Totales'},
             hole = 0.3,
             color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(textinfo='percent+label')

# Mostrar el gráfico
fig.show()

## 2. Tendencia y Estacionalidad:

### a) Todas las ventas del país con menos ventas

País con menos ventas:

In [ ]:
# Encontramos el país con menos ventas
paisMenosVentas = ventasPais.loc[ventasPais['AMOUNT'].idxmin()]
print(paisMenosVentas)

COUNTRY             Spain
SCENARIO           actual
AMOUNT      8131265.97752
Name: 17, dtype: object


In [ ]:
# Filtrar datos para el país con menos ventas y escenario 'actual'
datosSpainActual = data[(data['COUNTRY'] == paisMenosVentas['COUNTRY']) & (data['SCENARIO'] == 'actual')]

# Sumar el número de ventas
ventasSpainActual = datosSpainActual.groupby(['DATE', 'SCENARIO'])['AMOUNT'].sum().reset_index()

# Gráfico de Líneas
fig = px.line(ventasSpainActual, x='DATE', y='AMOUNT', color='SCENARIO',
              title=f'Ventas en {paisMenosVentas["COUNTRY"]}',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True
)

fig.show()

* **TENDENCIA:**

No se observa tendencia en las ventas de España, Las ventas en el mismo mes de los dos años son similares, por lo que ni aumentan ni disminuyen de 2023 a 2024.

* **ESTACIONALIDAD:**

Sí podemos oberservar una estacionalidad anual.

### b) Marca más vendida:

In [ ]:
datosPaisMenosVentas = data[data['COUNTRY'] == paisMenosVentas['COUNTRY']]

# Agrupar por 'SUBBRAND' y calcular el total de ventas para cada una
ventasPorSubmarca = datosPaisMenosVentas.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()


fig = px.pie( ventasPorSubmarca,
              names='SUBBRAND',
              values='AMOUNT',
              title=f'Distribución de Ventas Totales por Submarca en {paisMenosVentas["COUNTRY"]}',
              labels={'SUBBRAND': 'Submarca', 'AMOUNT': 'Ventas Totales'},
              hole=0.3,
              color_discrete_sequence=px.colors.qualitative.Pastel2
)

fig.show()

# Marca más vendida
submarcaMasVendida = ventasPorSubmarca.loc[ventasPorSubmarca['AMOUNT'].idxmax()]
print(submarcaMasVendida)

SUBBRAND     Pepsi Max (L3)
AMOUNT      85251053.001145
Name: 3, dtype: object


In [ ]:
# Filtrar los datos para la marca más vendida en el país con menos ventas y el escenario 'actual'
datosMarcaMasVendidaActual = datosPaisMenosVentas[
    (datosPaisMenosVentas['SCENARIO'] == 'actual') &
    (datosPaisMenosVentas['SUBBRAND'] == 'Pepsi Max (L3)')
]

# Agrupar por mes y sumar las ventas
ventasPorMesActual = datosMarcaMasVendidaActual.groupby('DATE')['AMOUNT'].sum().reset_index()


fig = px.line(
    ventasPorMesActual,
    x='DATE',
    y='AMOUNT',
    title=f'Tendencia de Ventas Mensuales de {submarcaMasVendida["SUBBRAND"]}',
    labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
    markers=True
)

# Mostrar el gráfico
fig.show()

* **TENDENCIA:**

No se observa tendencia en las ventas de Pepsi Max, Las ventas en el mismo mes de los dos años son similares, por lo que ni aumentan ni disminuyen de 2023 a 2024.

* **ESTACIONALIDAD:**

Sí podemos oberservar una estacionalidad anual.

## 3. Cuáles son las predicciones hechas en España y cómo de buenas son


Primero vamos a ver cómo son todas las predicciones hechas, de todas las marcas, posteriormente iremos especificando para poder analizarlas con mayor precisión.

In [ ]:
# Sacamos las predicciones hechas en España
datosPaisForecast = data[(data['COUNTRY'] == 'Spain') &
                         (data['SCENARIO'] == 'AI_forecast')
                         ]
# Sumar el número de ventas
datosPaisForecastVentas = datosPaisForecast.groupby('DATE')['AMOUNT'].sum().reset_index()


# Gráfico de Líneas
fig = px.line(datosPaisForecastVentas, x='DATE', y='AMOUNT',
              title= 'Predicciones Totales en España',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True
)

fig.show()

Como podemos observar, sumar todas las predicciones no nos da información relevante, de hecho, esta alterada debido a que hay más predicciones hechas para los meses de verano que para los meses de invierno.

Vamos a analizar las 12 predicciones por separado para poder analizar con mayor profundidad.

In [ ]:
# Filtrar los datos para España y el escenario 'AI_forecast'
datosPaisForecast = data[(data['COUNTRY'] == 'Spain') & (data['SCENARIO'] == 'AI_forecast')]

# Agrupar los valores de 'AMOUNT' por 'DATE' y 'FORECAST' y sumar
suma_por_forecast = datosPaisForecast.groupby(['DATE', 'FORECAST'])['AMOUNT'].sum().reset_index()

# Gráfico de Líneas con una línea para cada categoría de 'FORECAST'
fig = px.line(suma_por_forecast, x='DATE', y='AMOUNT', color='FORECAST',
              title='Predicciones en España',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True)

# Mostrar el gráfico
fig.show()

Observando las diferentes predicciones vemos que hay mucha diferencia en cuanto
las predicciones de los 3 primeros meses respecto a los demás.

Vamos a analizar esta diferencia, comparando las predicciones con el valor real de ventas.

In [ ]:
datosPaisForecast = data[(data['COUNTRY'] == 'Spain') &
                         (data['SCENARIO'] == 'AI_forecast') &
                         (data['FORECAST'] == 'AI_PF')]


# Sumar los valores de 'AMOUNT' agrupando por 'SUBBRAND'
datosPaisForecast = datosPaisForecast.groupby('DATE')['AMOUNT'].sum().reset_index()

datosPaisForecast['Tipo'] = 'Predicciones'  # Añadir una columna para distinguir el tipo de datos


ventasSpainActual['Tipo'] = 'Ventas Reales'  # Añadir una columna para distinguir el tipo de datos

# Combinar ambos DataFrames en uno solo
datos_combinados = pd.concat([ventasSpainActual, datosPaisForecast])

# Graficar ambas series en un gráfico de líneas
fig = px.line(datos_combinados, x='DATE', y='AMOUNT', color='Tipo',
              title='Predicciones de enero de 2023 y Ventas Reales en España',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True)

fig.show()

In [ ]:
datos_combinados = pd.merge(ventasSpainActual[['DATE', 'AMOUNT']],
                            datosPaisForecast[['DATE', 'AMOUNT']],
                            on='DATE', suffixes=('_real', '_pred'))

# Calcular el Error Cuadrático Medio (MSE)
mse = mean_squared_error(datos_combinados['AMOUNT_real'], datos_combinados['AMOUNT_pred'])
rmse = mse ** 0.5

mean_real = datos_combinados['AMOUNT_real'].mean()

error_percentage = (rmse / mean_real) * 100
print(f'Porcentaje de Error: {error_percentage:.2f}%')

Porcentaje de Error: 13.82%


Las prdicciones hechas en **enero de 2023** tiene un 13.82% de error, es decir, hay notables desviacíaciones. No obastante, como podemos observar en la gráfica, las predicciones siguen la tendencia de los valores reales.

Vamos a repetir el proceso con la predicción hecha en junio de 2023:

In [ ]:
datosPaisForecast = data[(data['COUNTRY'] == 'Spain') &
                         (data['SCENARIO'] == 'AI_forecast') &
                         (data['FORECAST'] == 'AI_P06F')]


# Sumar los valores de 'AMOUNT' agrupando por 'SUBBRAND'
datosPaisForecast = datosPaisForecast.groupby('DATE')['AMOUNT'].sum().reset_index()

datosPaisForecast['Tipo'] = 'Predicciones'  # Añadir una columna para distinguir el tipo de datos


ventasSpainActual['Tipo'] = 'Ventas Reales'  # Añadir una columna para distinguir el tipo de datos

# Combinar ambos DataFrames en uno solo
datos_combinados = pd.concat([ventasSpainActual, datosPaisForecast])

# Graficar ambas series en un gráfico de líneas
fig = px.line(datos_combinados, x='DATE', y='AMOUNT', color='Tipo',
              title='Predicciones de junio de 2023 y Ventas Reales en España',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True)

fig.show()

In [ ]:
datos_combinados = pd.merge(ventasSpainActual[['DATE', 'AMOUNT']],
                            datosPaisForecast[['DATE', 'AMOUNT']],
                            on='DATE', suffixes=('_real', '_pred'))

# Calcular el Error Cuadrático Medio (MSE)
mse = mean_squared_error(datos_combinados['AMOUNT_real'], datos_combinados['AMOUNT_pred'])
rmse = mse ** 0.5

mean_real = datos_combinados['AMOUNT_real'].mean()

error_percentage = (rmse / mean_real) * 100
print(f'Porcentaje de Error: {error_percentage:.2f}%')

Porcentaje de Error: 175.48%


Las prdicciones hechas en **junio de 2023** tiene un 78.82% de error, es decir, hay muchas desviacíaciones. Además, vemos que la tendencia no es la misma, por lo que es una pésima predicción.

**CONLCUSIONES PREDICCIONES TOTALES:**

Las predicciones hechas los 3 primeros meses, se adecuan a la realidad y pueden ser fiables. Sin embargo, los 9 siguientes meses, son demasiados optimistas y no se parecen en nada a los valores reales. Algo que no tiene sentido debido a que se han hecho más tarde que las primeras 3, por lo que tienen más información.

Lo que hemos podido ver es que hay más de una predicción hecha para cada mes, y en la práctica sumamos esos amounts para poder trabajar con ellos. Por lo que ahora vamos a repetir este último proceso, pero vamos a calcular la media de los amounts para los meses que tengan más de una predicción.  

In [ ]:
# Filtrar los datos para España y el escenario 'AI_forecast'
datosPaisForecast = data[(data['COUNTRY'] == 'Spain') &
                         (data['SCENARIO'] == 'AI_forecast')]

# Agrupar los valores de 'AMOUNT' por 'DATE' y 'FORECAST' y sumar
suma_por_forecast = datosPaisForecast.groupby(['DATE', 'FORECAST'])['AMOUNT'].mean().reset_index()

# Gráfico de Líneas con una línea para cada categoría de 'FORECAST'
fig = px.line(suma_por_forecast, x='DATE', y='AMOUNT', color='FORECAST',
              title='Predicciones en España',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True)

# Mostrar el gráfico
fig.show()

In [ ]:
datosPaisForecast = data[(data['COUNTRY'] == 'Spain') &
                         (data['SCENARIO'] == 'AI_forecast') &
                         (data['FORECAST'] == 'AI_PF')]


# Sumar los valores de 'AMOUNT' agrupando por 'SUBBRAND'
datosPaisForecast = datosPaisForecast.groupby('DATE')['AMOUNT'].mean().reset_index()

datosPaisForecast['Tipo'] = 'Predicciones'  # Añadir una columna para distinguir el tipo de datos


ventasSpainActual['Tipo'] = 'Ventas Reales'  # Añadir una columna para distinguir el tipo de datos

# Combinar ambos DataFrames en uno solo
datos_combinados = pd.concat([ventasSpainActual, datosPaisForecast])

# Graficar ambas series en un gráfico de líneas
fig = px.line(datos_combinados, x='DATE', y='AMOUNT', color='Tipo',
              title='Predicciones de enero de 2023 y Ventas Reales en España',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True)

fig.show()

In [ ]:
datos_combinados = pd.merge(ventasSpainActual[['DATE', 'AMOUNT']],
                            datosPaisForecast[['DATE', 'AMOUNT']],
                            on='DATE', suffixes=('_real', '_pred'))

# Calcular el Error Cuadrático Medio (MSE)
mse = mean_squared_error(datos_combinados['AMOUNT_real'], datos_combinados['AMOUNT_pred'])
rmse = mse ** 0.5

mean_real = datos_combinados['AMOUNT_real'].mean()

error_percentage = (rmse / mean_real) * 100
print(f'Porcentaje de Error: {error_percentage:.2f}%')

Porcentaje de Error: 83.40%


**CONCLUSIONES HACIENDO LA MEDIA:**

Hemos podido comprobar que hacer la media de los AMOUNTS no ha sido una buena idea, debido a que el error es mayor en todas las predicciones. Por lo que nos quedamos con las conclusiones anteriores.


**Las predicciones hechas los 3 primeros meses, se adecuan a la realidad y pueden ser fiables. Sin embargo, los 9 siguientes meses, son demasiados optimistas y no se parecen en nada a los valores reales.**

  Ahora vamos a ver si las predicciones se adecuan más si las obervamos por las ventas de las diferentes marcas de manera individual.

In [ ]:
# Datos de la primera predicción de PEPSI MAX
datosSpainForecastPEPSI = data[(data['COUNTRY'] == 'Spain') &
                         (data['SCENARIO'] == 'AI_forecast') &
                         (data['SUBBRAND'] == 'Pepsi Max (L3)') &
                         (data['FORECAST'] == 'AI_PF')]
datosSpainForecastPEPSI = datosSpainForecastPEPSI.groupby('DATE')['AMOUNT'].sum().reset_index()

# Datos de ventas reales de PEPSI MAX
datosSpainPEPSI = data[(data['COUNTRY'] == 'Spain') &
                         (data['SCENARIO'] == 'actual') &
                         (data['SUBBRAND'] == 'Pepsi Max (L3)')]
datosSpainPEPSI = datosSpainPEPSI.groupby('DATE')['AMOUNT'].sum().reset_index()

datosSpainForecastPEPSI['Tipo'] = 'Predicciones'  # Añadir una columna para distinguir el tipo de datos

datosSpainPEPSI['Tipo'] = 'Ventas Reales de PEPSI MAX'  # Añadir una columna para distinguir el tipo de datos

# Combinar ambos DataFrames en uno solo
datos_combinados = pd.concat([datosSpainPEPSI, datosSpainForecastPEPSI])

# Graficar ambas series en un gráfico de líneas
fig = px.line(datos_combinados, x='DATE', y='AMOUNT', color='Tipo',
              title='Predicciones de enero de 2023 y Ventas Reales en España de PEPSI MAX ',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True)

fig.show()

In [ ]:
datos_combinados = pd.merge(datosSpainPEPSI[['DATE', 'AMOUNT']],
                            datosSpainForecastPEPSI[['DATE', 'AMOUNT']],
                            on='DATE', suffixes=('_real', '_pred'))

# Calcular la diferencia absoluta entre las ventas reales y las predicciones, luego los porcentajes
datos_combinados['Diferencia'] = abs(datos_combinados['AMOUNT_real'] - datos_combinados['AMOUNT_pred'])
datos_combinados['Porcentaje_Diferencia'] = (datos_combinados['Diferencia'] / datos_combinados['AMOUNT_real']) * 100

# Valor medio del porcentaje de diferencia
valor_medio_porcentaje = datos_combinados['Porcentaje_Diferencia'].mean()
print("Valor medio del porcentaje de diferencia:", valor_medio_porcentaje, "%")


# Crear gráfico de barras con Plotly
fig = px.line(datos_combinados, x='DATE', y='Porcentaje_Diferencia',
             title='Porcentaje de Diferencia entre Ventas Reales y Predicciones',
             labels={'Porcentaje_Diferencia': 'Porcentaje de Diferencia (%)', 'DATE': 'Fecha'},
             width = 1300
             )

fig.add_hline(y=100, line_dash="dash", line_color="red", annotation_text="100%", annotation_position="top right")

fig.show()

Valor medio del porcentaje de diferencia: 18.894353737993498 %


Las predicciones hechas en enero de 2023 desvían de media un 18.9% respecto a los valores reales. Podemos decir que no buenas predicciones, pero tampoco malas.

Vamos a hacer lo mismo con las predicciones de noviembre, que por lo que ye sabemos son bastante diferentes a las de lso 3 primeros meses.

In [ ]:
# Datos de la primera predicción de PEPSI MAX
datosSpainForecastPEPSI = data[(data['COUNTRY'] == 'Spain') &
                         (data['SCENARIO'] == 'AI_forecast') &
                         (data['SUBBRAND'] == 'Pepsi Max (L3)') &
                         (data['FORECAST'] == 'AI_P11F')]
datosSpainForecastPEPSI = datosSpainForecastPEPSI.groupby('DATE')['AMOUNT'].sum().reset_index()

# Datos de ventas reales de PEPSI MAX
datosSpainPEPSI = data[(data['COUNTRY'] == 'Spain') &
                         (data['SCENARIO'] == 'actual') &
                         (data['SUBBRAND'] == 'Pepsi Max (L3)')]
datosSpainPEPSI = datosSpainPEPSI.groupby('DATE')['AMOUNT'].sum().reset_index()

datosSpainForecastPEPSI['Tipo'] = 'Predicciones'  # Añadir una columna para distinguir el tipo de datos

datosSpainPEPSI['Tipo'] = 'Ventas Reales de PEPSI MAX'  # Añadir una columna para distinguir el tipo de datos

# Combinar ambos DataFrames en uno solo
datos_combinados = pd.concat([datosSpainPEPSI, datosSpainForecastPEPSI])

# Graficar ambas series en un gráfico de líneas
fig = px.line(datos_combinados, x='DATE', y='AMOUNT', color='Tipo',
              title='Predicciones de noviembre 023 y Ventas Reales en España de PEPSI MAX ',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True)

fig.show()

In [ ]:
datos_combinados = pd.merge(datosSpainPEPSI[['DATE', 'AMOUNT']],
                            datosSpainForecastPEPSI[['DATE', 'AMOUNT']],
                            on='DATE', suffixes=('_real', '_pred'))

# Calcular la diferencia absoluta entre las ventas reales y las predicciones, luego los porcentajes
datos_combinados['Diferencia'] = abs(datos_combinados['AMOUNT_real'] - datos_combinados['AMOUNT_pred'])
datos_combinados['Porcentaje_Diferencia'] = (datos_combinados['Diferencia'] / datos_combinados['AMOUNT_real']) * 100

# Valor medio del porcentaje de diferencia
valor_medio_porcentaje = datos_combinados['Porcentaje_Diferencia'].mean()
print("Valor medio del porcentaje de diferencia:", valor_medio_porcentaje, "%")

# Crear gráfico de barras con Plotly
fig = px.line(datos_combinados, x='DATE', y='Porcentaje_Diferencia',
             title='Porcentaje de Diferencia entre Ventas Reales y Predicciones',
             labels={'Porcentaje_Diferencia': 'Porcentaje de Diferencia (%)', 'DATE': 'Fecha'},
             width = 1300
             )

fig.add_hline(y=100, line_dash="dash", line_color="red", annotation_text="100%", annotation_position="top right")

fig.show()

Valor medio del porcentaje de diferencia: 337.31808966988024 %


El porcentaje de error see eleva a más del 300%, por lo que las predicciones son pésimas, justo lo mismo que pasaba obersvando las predicciones totales.